Initialize the layered graph

In [95]:
def createLayeredGraph(G, H):
    V, E = G  # Unpack the original graph
    V_prime, E_prime = set(), set()  # Initialize empty sets for the layered graph

    # Expand vertices into layers, starting from hop count 0
    for v in V:
        for i in range(0, H + 1):  # Include H, since we're starting from 0
            V_prime.add((v, i))

    # Connect vertices across layers based on original edges, considering hop counts
    for (u, v) in E:
        for i in range(0, H):  # Up to H-1 since we're starting from 0
            E_prime.add(((u, i), (v, i + 1)))

    return (V_prime, E_prime)  # Return the layered graph

Test the initialization

In [96]:
# Define the original graph G
V = {0, 1, 2, 3}
E = {(0, 1), (0, 2), (1, 3), (2, 3)}
G = (V, E)
H = 2

# Create the layered graph
G_prime = createLayeredGraph(G, H)

# Display the results
print("Layered Vertices V':\n", G_prime[0])
print("Layered Edges E':\n", G_prime[1])

Layered Vertices V':
 {(0, 1), (1, 2), (2, 1), (0, 0), (3, 1), (1, 1), (2, 0), (3, 0), (0, 2), (2, 2), (1, 0), (3, 2)}
Layered Edges E':
 {((2, 0), (3, 1)), ((1, 1), (3, 2)), ((0, 0), (2, 1)), ((0, 0), (1, 1)), ((0, 1), (1, 2)), ((1, 0), (3, 1)), ((2, 1), (3, 2)), ((0, 1), (2, 2))}


In [97]:
# Function to add new edges from source to corresponding source nodes and from destination to corresponding destination nodes
def add_source_and_dest_to_G_prime(G_prime, source, destination):
    
    
    (V_prime, E_prime) = G_prime
    
    # We call -1 the new layer introduced to handle source and destination
    V_prime.add((source, -1))
    V_prime.add((destination, -1))

    # Add new edges from source to corresponding source nodes in each layer
    for i in range(0, H+1):
        E_prime.add(((source, -1), (source, i)))

    # Add new edges from destination to corresponding destination nodes in each layer
    for i in range(0, H+1):
        E_prime.add(((destination, i), (destination, -1)))

    return (V_prime, E_prime)

Test G_prime with also the newly added source and destination

In [98]:
G_prime = add_source_and_dest_to_G_prime(G_prime, 0, 3)

# Display the results
print("Layered Vertices V':\n", G_prime[0])
print("Layered Edges E':\n", G_prime[1])

Layered Vertices V':
 {(0, 1), (1, 2), (3, -1), (2, 1), (0, 0), (3, 1), (1, 1), (2, 0), (3, 0), (0, 2), (2, 2), (1, 0), (3, 2), (0, -1)}
Layered Edges E':
 {((2, 0), (3, 1)), ((1, 1), (3, 2)), ((0, 0), (2, 1)), ((3, 0), (3, -1)), ((0, 0), (1, 1)), ((3, 1), (3, -1)), ((3, 2), (3, -1)), ((0, 1), (1, 2)), ((0, -1), (0, 0)), ((0, -1), (0, 2)), ((1, 0), (3, 1)), ((2, 1), (3, 2)), ((0, -1), (0, 1)), ((0, 1), (2, 2))}


In [99]:
from collections import deque

def delete_unreachable_nodes(G_prime, source):
    V_prime, E_prime = G_prime  # Unpack the layered graph
    reachable_nodes = set()  # Set to keep track of reachable nodes
    reachable_edges = set()  # Set to keep track of edges connecting reachable nodes
    queue = deque([(source, -1)])  # Initialize a queue with the source node, using deque for efficiency

    # Perform a Breadth-First Search (BFS) to find all reachable nodes from the source
    while queue:
        current_node = queue.popleft()  # Get the first node in the queue efficiently
        if current_node not in reachable_nodes:
            reachable_nodes.add(current_node)  # Mark the current node as reachable
            # Iterate over edges to find and enqueue directly reachable nodes
            for (u, v) in E_prime:
                if u == current_node:
                    reachable_edges.add((u, v))  # Keep track of the reachable edge
                    if v not in reachable_nodes and v not in queue:  # Avoid duplicates in the queue
                        queue.append(v)

    # The filtered graph consists of reachable nodes and edges between them
    V_prime_filtered = reachable_nodes
    E_prime_filtered = reachable_edges

    return (V_prime_filtered, E_prime_filtered)

In [100]:
G_prime = delete_unreachable_nodes(G_prime, 0)

# Display the results
print("Layered Vertices without unreachable nodes V':\n", G_prime[0])
print("Layered Edges without unreachable nodes E':\n", G_prime[1])

Layered Vertices without unreachable nodes V':
 {(0, 1), (1, 2), (3, -1), (2, 1), (0, 0), (1, 1), (0, 2), (2, 2), (3, 2), (0, -1)}
Layered Edges without unreachable nodes E':
 {((1, 1), (3, 2)), ((0, 0), (2, 1)), ((0, 0), (1, 1)), ((3, 2), (3, -1)), ((0, 1), (1, 2)), ((0, -1), (0, 0)), ((0, -1), (0, 2)), ((2, 1), (3, 2)), ((0, -1), (0, 1)), ((0, 1), (2, 2))}


In [101]:
def delete_nodes_not_leading_to_destination(G_prime, destination):
    V_prime, E_prime = G_prime  # Unpack the graph into vertices and edges
    reachable_nodes = set()  # Set to keep track of nodes leading to the destination
    reachable_edges = set()  # Set to keep track of edges leading to the destination
    queue = deque([(destination, -1)])  # Initialize a queue with the destination node

    # Perform a reverse Breadth-First Search (BFS) from the destination
    while queue:
        current_node = queue.popleft()  # Get the first node in the queue
        if current_node not in reachable_nodes:
            reachable_nodes.add(current_node)  # Mark the current node as leading to the destination

            # Iterate over edges in reverse to find and enqueue nodes leading to the current node
            for (u, v) in E_prime:
                if v == current_node:
                    reachable_edges.add((u, v))  # Keep track of the edge leading to the destination
                    if u not in reachable_nodes and u not in queue:  # Avoid duplicates in the queue
                        queue.append(u)

    # The filtered graph consists of nodes and edges leading to the destination
    V_prime_filtered = reachable_nodes
    E_prime_filtered = reachable_edges

    return (V_prime_filtered, E_prime_filtered)



In [102]:
G_prime = delete_nodes_not_leading_to_destination(G_prime, 3)

# Display the results
print("Layered Vertices without unreachable nodes V':\n", G_prime[0])
print("Layered Edges without unreachable nodes E':\n", G_prime[1])

Layered Vertices without unreachable nodes V':
 {(3, -1), (2, 1), (0, 0), (1, 1), (3, 2), (0, -1)}
Layered Edges without unreachable nodes E':
 {((1, 1), (3, 2)), ((0, 0), (2, 1)), ((3, 2), (3, -1)), ((0, 0), (1, 1)), ((0, -1), (0, 0)), ((2, 1), (3, 2))}


In [103]:
def assign_incoming_edges(G_prime):
    V_prime, E_prime = G_prime
    # Initialize a dictionary to hold the list of incoming edges for each node
    incoming_edges = {node: [] for node in V_prime}

    # Iterate through each edge in E_prime
    for (u, v) in E_prime:
        # Add the edge to the list of incoming edges for the destination node
        if v in incoming_edges:  # Ensure the destination node is in the graph
            incoming_edges[v].append((u, v))

    return incoming_edges

In [105]:
# Test assign incoming edges function
incoming_edges = assign_incoming_edges(G_prime)

for node, edges in incoming_edges.items():
    print(f"Node {node} has incoming edges: {edges}")

Node (3, -1) has incoming edges: [((3, 2), (3, -1))]
Node (2, 1) has incoming edges: [((0, 0), (2, 1))]
Node (0, 0) has incoming edges: [((0, -1), (0, 0))]
Node (1, 1) has incoming edges: [((0, 0), (1, 1))]
Node (3, 2) has incoming edges: [((1, 1), (3, 2)), ((2, 1), (3, 2))]
Node (0, -1) has incoming edges: []


In [ ]:
def is_path_operational(path):
    '''randomly returns true with the probability below'''
    probability = 0.7
    return random.random() < probability
    

In [38]:
import random
import math
import numpy as np

class MAB:
    def __init__(self, n_arms, epsilon=0.3):
        self.n_arms = n_arms  # Number of arms
        self.epsilon = epsilon  # Exploration rate
        self.arm_counts = np.zeros(n_arms)  # Count of pulls for each arm
        self.arm_loss = np.zeros(n_arms)  # Total loss for each arm

    def select_arm(self):
        """Selects an arm based on the ε-greedy strategy."""
        if np.random.rand() < self.epsilon:  # Explore
            chosen_arm = np.random.randint(self.n_arms)
            print(f"Exploring: Selected arm {chosen_arm}")
        else:  # Exploit
            # Select the arm with the minimum average loss
            avg_losses = self.arm_loss / (self.arm_counts + 1e-5)  # Adding a small value to avoid division by zero
            chosen_arm = np.argmin(avg_losses)
            print(f"Exploiting: Selected arm {chosen_arm} with min avg loss")
        return chosen_arm

    def update(self, chosen_arm, loss):
        """Updates the estimated losses and counts for the chosen arm."""
        self.arm_counts[chosen_arm] += 1
        self.arm_loss[chosen_arm] += loss
        print(f"Updated losses for arm {chosen_arm}: Total loss = {self.arm_loss[chosen_arm]}, Pull count = {self.arm_counts[chosen_arm]}")

        
class Node:
    def __init__(self, id, arms, q):
        self.id = id
        self.mab = MAB(q, arms)  # Initialize MAB with arms corresponding to incoming edges
        self.path_from_root = []

    def select_edge(self):
        """Selects an incoming edge based on the MAB algorithm."""
        return self.mab.select_arm()

    def update_mab(self, chosen_arm, reward):
        """Updates the MAB based on the outcome of the chosen edge."""
        self.mab.update(chosen_arm, reward)    
        
        

def generate_spanning_tree_W(nodes):
    spanning_tree = set()
    for node in nodes:
        selected_edge = node.select_edge()
        spanning_tree.add(selected_edge)
    return spanning_tree
    

def exploration_step(nodes, current_phase, q, s, r):
    
    # select a random node v
    v = random.choice(nodes)
    
    # we build the path from s to r by getting P(Wτi , v) then we select any path to reach r
    path_s_to_v = v.path_from_root
    # path_v_to_r = generate_path_v_to_r(v, r)
    
    Bcost = 0 if is_path_operational(path_s_to_v) else 1
    
    current_phase['time_steps'][v.id].append(Bcost)
    
def exploitation_step(nodes, current_phase, s, r):
    path_s_to_r = r.path_from_root


def generate_path_from_root(spanning_tree, node):
    parent_mapping = {}
    for parent, child in spanning_tree:
        parent_mapping[child] = parent

    path = deque()  

    # Trace back from the target node to the root using the parent mapping
    current_node = node.id
    while current_node in parent_mapping:
        parent = parent_mapping[current_node]
        path.appendleft((parent, current_node))  # Append to the left side of the deque
        current_node = parent

    return list(path) 
    

def simulate_phase(nodes, q, s, r, lambda_phase_length):
    
    # Beginning of a phase, each phase is characterized by a spanning tree and a 
    current_phase = {'spanning_tree': generate_spanning_tree_W(nodes), 'time_steps': {}}
    # Generating for every node in the current phase the path to origin P(W_tau_i, v)
    for node in nodes:
        node.path_from_root = generate_path_from_root(node, current_phase)
    
    # During a phase we do exploration and exploitation
    for t in range(lambda_phase_length):
        if random.random() < q:
            exploration_step(nodes, current_phase, q, s, r)
        else:
            exploitation_step(nodes, current_phase, s, r)
    
    # End of phase
    for node in nodes:
        selected_time_step = random.choice(current_phase['time_steps'][node.id])
        node.update_mab(node.path_from_root[-1], selected_time_step)
            
    return current_phase
    


def E2EA_algorithm(G_prime, q, s, r, T):
    V_prime, E_prime = G_prime
    n = len(V_prime)
    lambda_phase_length = int((n / q) * math.log(n))
    
    # initialize the MAB of the nodes
    nodes = []
    nodes_incoming_edges = assign_incoming_edges(G_prime) # generate a dictionary with the incoming edge of each node
    for node_id in V_prime:
        node = Node(node_id, nodes_incoming_edges[node_id], q)
        nodes.append(node)
    
    
    # run the phases of the algorithm
    for _ in range(T // lambda_phase_length):
        simulate_phase(nodes, q, s, r)
    return